### Number Function for XOPT

In [1]:
# import libraries
import sys 
import os 
import numpy as np 
import matplotlib.pyplot as plt 
from os.path import basename
from re import sub
import import_ipynb
from SimpleSource_TRAK_tools_2_CHANGE_FE_2 import *
import subprocess
import statistics
from SimpleSource_TRAK_build_analyze_tools_2_CHANGE_FE_2 import *

import time 
from random import random 
from typing import Dict
from xopt.vocs import VOCS

importing Jupyter notebook from SimpleSource_TRAK_tools_2_CHANGE_FE.ipynb
importing Jupyter notebook from SimpleSource_TRAK_build_analyze_tools_2_CHANGE_FE.ipynb
importing Jupyter notebook from SimpleSource_TRAK_tools_2.ipynb


In [ ]:
# create the VOCS 
TRAK_MK_vocs = VOCS(
    **{
        "variables": {"an_x": [43, 90], "an_y": [29, 39],  "fe_x": [0, 40],  "fe_y": [13, 37], "fe_pot": [-1860, -1360]},
        "objectives": {"Jz_range_max": "MINIMIZE", "I_dens_dev": "MINIMIZE"},
        "constraints": {"E_max": ["LESS_THAN", 110], }
    }
)

In [ ]:
# number function for optimization
#def TRAK_FUNCTION(an_x, an_y, fe_x, fe_y, fe_pot, subdir):  ## edit how we input the variables here! 
def TRAK_FUNCTION(individual):
    print('\n START OF NEW SOURCE: ')
    an_x = individual[0]
    an_y = individual[1]
    fe_x = individual[2]
    fe_y = individual[3]
    fe_pot = individual[4]
    subdir = individual[5]
    
    print('anx:', an_x, 'any:', an_y, 'fex:', fe_x, 'fey:', fe_y, 'fepot:', fe_pot)
    
    # get strings of inputs (determine how precise we want to be about the values we go out to )
    fe_x = np.round(fe_x, 0)
    fe_y = np.round(fe_y, 0)
    an_x = np.round(an_x, 0)
    an_y = np.round(an_y, 0)
    fe_pot = np.round(fe_pot, 0)
    
    print('an_x: ', an_x, 'an_y: ', an_y, 'fe_x: ', fe_x, 'fe_y: ', fe_y, 'fe_pot: ', fe_pot)
    
    fe_x_str = str(fe_x).replace('.', '')
    fe_y_str = str(fe_y).replace('.', '')
    an_x_str = str(an_x).replace('.', '')
    an_y_str = str(an_y).replace('.', '')
    fe_pot_str = str(fe_pot).replace('.', '')
    fe_pot_str2 = str(fe_pot)

    ## inputs for getting estat solution
    meshfile_estat = '%s/mesh_%s_%s_%s_%s_%s' %(subdir, fe_x_str, fe_y_str, an_x_str, an_y_str, fe_pot_str) # define mesh file for estat
    estatfile = '%s/estat_%s_%s_%s_%s_%s' %(subdir, fe_x_str, fe_y_str, an_x_str, an_y_str, fe_pot_str) # define estat file 
    potential = '-1.36E+03' # define the potential for the estat solution

    ## inputs for getting permag solution
    meshfile_permag = '%s/mesh_permag' %subdir # define the mesh file for permag
    permagfile = '%s/permag' %subdir # define permag file

    ## inputs for getting TRAK base solution -- change to be right!! 
    trakfile = '%s/firstfile' %subdir
    
    print(meshfile_estat)
    print(estatfile)
    print(meshfile_permag)
    print(permagfile)
    print(trakfile)
    
    # Getting estat solution: 
    # create and run files (.min, .mls, .mou, .ein, .els, .eou)
    run_mesh_estat(fe_x, fe_y, an_x, an_y, meshfile_estat)  # make and run the mesh for estat
    run_estat(estatfile, meshfile_estat, potential, fe_pot_str2) # make and run the .ein file for estat
    
    # Getting permag solution:
    # create and run files (.min, .mls, .mou, .pin, .pls, .pou)
    run_mesh_permag(meshfile_permag) # make and run the mesh for permag
    run_permag(permagfile, meshfile_permag) # make and run the .pin for permag
    
    # Getting TRAK base solution:
    # create and run files (.tin, .tls, .tou)
    run_trak(estatfile, permagfile, trakfile)
    
    ## Cartesian product of avg and demit 
    #avg_list = ['0.08', '0.12', '0.16', '0.20', '0.24']
    #demit_list = ['0.10', '0.15', '0.2', '0.25', '0.3']
    
    #avg_list = ['0.08', '0.16', '0.24']
    #demit_list = ['0.10', '0.2', '0.3']
    
    #avg_list = ['0.08', '0.24']
    #demit_list = ['0.10', '0.30']
    
    avg_list = ['0.08']
    demit_list = ['0.10']
    file_list = []
    for i in avg_list: 
        avg_float = float(i)
        avg_file = i.replace('.', '')
        for j in demit_list: 
            demit_float = float(j)
            demit_file = j.replace('.', '')
            avg_demit_ncycle_func('%s' %trakfile,
                                  '%s/a%sd%s_%s_%s_%s_%s_%s' %(subdir, avg_file, demit_file, fe_x_str, fe_y_str, an_x_str, an_y_str, fe_pot_str), 
                                 avg_float, demit_float, 60)
            file_list.append('%s/a%sd%s_%s_%s_%s_%s_%s' %(subdir, avg_file, demit_file, fe_x_str, fe_y_str, an_x_str, an_y_str, fe_pot_str))

    print('Running files: ', file_list)
    
    ## run all the files
    run_parallel(file_list, '3', 'TRAK')
    
    print('Running files complete: ', file_list)
    
    ##### Analyze Simulations: ################################################################################################################
    # Set conditions and analysis parameters
    conv_condition = 0.6
    density_slice = 150
    radius = 8.7

    # get list of files where the current converges. (something is going wrong here...)
    print(file_list)
    print(conv_condition)
    convergence = converg_func(file_list, conv_condition)
    print(convergence)
    converging_file_list = convergence[1]

    # get and plot the current convergence of the converging simulations 
    current_list = []
    converging_list_base = []#
    for i in converging_file_list: 
        converging_list_base.append(i[0])
        current_list.append(current_conv(i[0]))
        plt.legend().remove()

    mean = np.mean(current_list)
    stdev = np.std(current_list)
    print('I = %.4f \u00B1 %.4f A' %(mean, stdev))
    plt.show()
    
    ## Get and plot the current density of the converging simulations
    #current_list = []
    for i in converging_file_list: 
       cdensity(i[0], density_slice)
    plt.xlim(0, 20)
    plt.axvline(8.7)
    #plt.ylim(0, 150)
    plt.show()

    ## the flatness analysis
    flatness_analysis = flatness(converging_list_base, density_slice, radius)
    cdens_list = []
    Jz_range_list = []
    for i in flatness_analysis: 
        cdens_list.append(i[1])
        Jz_range_list.append(i[2])

    cdens_avg = np.mean(cdens_list)
    Jz_range_max = max(Jz_range_list)

    print("current density mean: ", cdens_avg, 'A/m^2')
    print("max Jz range: ", Jz_range_max, 'A/m^2')
    cdens_avg_dev = abs(cdens_avg - 9)  # deviation from goal current density 
    print("current density mean:", cdens_avg)
    print("current density mean (deviation from 9): ", cdens_avg_dev)
    
    ## Get the electric field results: 
    # lines where the field was calculated 
    #E_field_line_loc = [-10, 0, 1.5, 3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 10]
    E_field_line_loc = [-25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 31, 32, 33, 34, 35, 40, 45, 50, 55, 60, 65, 70, 75]

    #print(converging_file_list)
    max_e_fields = []
    e_field_values = []
    for i in converging_file_list:
        for val in E_field_line_loc:
            e_field_output = E_field_line(i[0], val, plots = False)
            max_e_fields.append([e_field_output[0], val])
            e_field_values.append(e_field_output[1])

    def Extract(first):
        return [item[0] for item in first]

    #print(max_e_fields)
    max_field_sim = max(Extract(max_e_fields))
    print('maximum electric field calculated: ', max_field_sim)
    
    #Getting the percentile stats of the distributions: 
    e_field_list = []
    for i in e_field_values: 
        for val in i: 
            e_field_list.append(val)
    #print(e_field_list)
    e_field_array = np.array(e_field_list)
    percentile_99 = np.percentile(e_field_array, 99)
    percentile_95 = np.percentile(e_field_array, 95)
    print("99th percentile value: ", percentile_99)
    print("95th percentile value: ", percentile_95)

    
    objectives = (Jz_range_max, cdens_avg_dev)
    constraints = (percentile_99, )

    
    return objectives, constraints

In [ ]:
def evaluate_TRAK_FUNCTION(
    inputs: Dict,
    sleep=0,
    random_sleep=0,
    raise_probability=0,
    **params
):
    subdir = 'xopt_simple_source_CHANGE_FE_3'
    ind = [inputs["an_x"], inputs["an_y"], inputs["fe_x"], inputs["fe_y"], inputs["fe_pot"], subdir]
    objectives, constraints = TRAK_FUNCTION(ind)
    
    r = random()
    # Sleep for a bit
    time.sleep(sleep)
    time.sleep(r * random_sleep * 2)  # Average should be random_sleep

    if r < raise_probability:
        raise ValueError("intentional TNK crash")
        
    outputs = {
        "Jz_range_max" : objectives[0],
        "I_dens_dev" : objectives[1],
        "E_max" : constraints[0], 
    }
    
    return outputs